In [1]:
import scipy.sparse as sparse
import implicit
import pandas as pd
import numpy as np
import json
import pickle

In [2]:
USER_ID = 'session'
ITEM_ID = 'aid'

In [3]:
raw_opt_path = '../../input/train_test/'
preprocess_path = '../../input/train_valid/'
output_path = '../../input/preprocess/'

SEED = 1208

In [4]:
use_all_data = True

# chris dataset check
if use_all_data == True:
    train = pd.read_parquet(raw_opt_path + 'train.parquet')
    test = pd.read_parquet(raw_opt_path + 'test.parquet')
else:
    train = pd.read_parquet(preprocess_path + 'train.parquet')
    test = pd.read_parquet(preprocess_path + 'test.parquet')
    
if use_all_data == True:
    prefix = 'test_'
else:
    prefix = 'train_'

In [5]:
%%time
df = pd.concat([train, test])
df['user_label'], user_idx = pd.factorize(df[USER_ID])
df['item_label'], item_idx = pd.factorize(df[ITEM_ID])
sparse_item_user = sparse.csr_matrix((np.ones(len(df)), (df['user_label'], df['item_label'])))
epoch, emb_size = 5000, 64
model = implicit.bpr.BayesianPersonalizedRanking(factors=emb_size, regularization=0.001, iterations=epoch,
                                                 random_state=SEED)
model.fit(sparse_item_user)
u2emb = dict(zip(user_idx, model.user_factors.to_numpy()))
i2emb = dict(zip(item_idx, model.item_factors.to_numpy()))

  0%|          | 0/5000 [00:00<?, ?it/s]

CPU times: user 46min 6s, sys: 55.5 s, total: 47min 2s
Wall time: 47min 4s


In [7]:
with open(f'{output_path}u2emb{prefix}.pkl', 'wb') as fp:
    pickle.dump(u2emb, fp)

In [8]:
with open(f'{output_path}i2emb{prefix}.pkl', 'wb') as fp:
    pickle.dump(i2emb, fp)